## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd

## Метрики 

1. Часы сна

2. Трудоустройство

3. Вес

4. Рост

5. Расстояние до ВУЗа в часах

6. Родители (мама, папа) что они пьют

7. кофе/чай

## Обработка данных

In [ ]:
df = pd.read_excel('data.xlsx')

df = df.rename(columns={
        'Часы сна':'hours_sleep',
        'Трудоустройство' : 'is_working',
        'Вес' :'weight',
        'Рост' :'height',
        'Расстояние до ВУЗа в часах' :'hours_road',
        'Родители (мама, папа) что они пьют' : 'parents',
        'кофе/чай' : 'res',
    })

df

,hours_sleep,is_working,weight,height,hours_road,parents,res
0,5,0,90,175,2.000000,кк,ч
1,5,1,82,172,0.800000,кч,ч
2,7,1,75,177,1.000000,кч,к
3,10,0,69,179,0.250000,чч,ч
4,5,1,100,180,2.000000,чч,ч
5,8,0,67,170,2.000000,кк,ч
6,6,1,65,179,1.500000,кч,к
7,8,0,88,187,0.170000,кк,ч
8,6,0,103,184,1.000000,чч,ч
9,8,1,85,195,0.666667,кк,к


#### Нужно перевести текстовые параметры в числовые

1. кофе/чай приводим в соответствии: 1 - кофе, 0 - чай.

2. Родители (мама, папа) что пьют = ['00', '01', '10', '11']

In [42]:
first_parents, second_parents, res = [],[],[]

for tup in df.itertuples():
    first_parents.append(10 if tup.parents[0] == 'к' else 0) 
    second_parents.append(10 if tup.parents[1] == 'к' else 0)
    res.append(10 if tup.res == 'к' else 0)
    
df['first_parent'] = first_parents
df['second_parent'] = second_parents
df['res'] = res
df = df.drop(columns=['parents'])

Чтобы классификация была точной, нужно привести данные фичей к близким значениям.

Иначе фичи, вроде first_parent, second_parent, hours_read, etc. не будут влиять на результат

In [ ]:
df['weight'] = df['weight'] / 10
df['height'] = df['height'] / 10
df['hours_road'] = df['hours_road'] * 5
df['is_working'] = df['is_working'] * 10

In [43]:
df

,hours_sleep,is_working,weight,height,hours_road,res,first_parent,second_parent
0,5,0,9.0,17.5,10.000000,0,10,10
1,5,10,8.2,17.2,4.000000,0,10,0
2,7,10,7.5,17.7,5.000000,10,10,0
3,10,0,6.9,17.9,1.250000,0,0,0
4,5,10,10.0,18.0,10.000000,0,0,0
5,8,0,6.7,17.0,10.000000,0,10,10
6,6,10,6.5,17.9,7.500000,10,10,0
7,8,0,8.8,18.7,0.850000,0,10,10
8,6,0,10.3,18.4,5.000000,0,0,0
9,8,10,8.5,19.5,3.333333,10,10,10


### подаём на вход:
1. что нужно определить
2. K-параметр
3. Готовый датасет метрик
4. Ответы

In [ ]:
X = []

for tup in df.itertuples():
    X.append([
        tup.hours_sleep,
        tup.is_working,
        tup.weight,
        tup.height,
        tup.hours_road,
        tup.first_parent,
        tup.second_parent
    ])
    
y = df['res'].to_list()

In [192]:
class KNN:
    def __init__(self, datasets, labels):
        self.datasets = datasets
        self.labels = labels
            
    def calc_proximity (self, firstObject : list, secondObject : list) -> float:
        if len(firstObject) != len(secondObject):
            raise ValueError('Ошибка загружаемых данных. Неверно заданы метрики')
        else:
            return ( sum( (item[0] - item[1]) ** 2 for item in zip(firstObject, secondObject) ) ** 0.5 ) 
    
    def calc_class (self, data:list, k:int) -> int:
        if len(data) == 0:
            raise ValueError('Ошибка загружаемых данных.')
        elif k < 2 and k >= len(self.datasets):
            raise ValueError('Число ближайших соседей должно быть больше одного и не равно общему числу соседей.')
        else:
            proxDataset = sorted( zip([ self.calc_proximity(data, dataset) for dataset in self.datasets ],  self.labels), key=lambda x: x[0])[:k]
            
            res = sum([item[1] for item in proxDataset]) / (k * 10)
            return 0 if res < 0.5 else 10

In [198]:
KNN(X, y).calc_class(X[18], 5)

0